In [33]:
import pandas as pd

In [34]:
# Change the working directory to the 'titanic' folder within the 'Kaggle Competitions' directory on the desktop\n",
import os 
os.chdir('/Users/willhughes/Desktop/Kaggle  Competitions/titanic')


In [35]:
# Load the training and test datasets from the Titanic Kaggle competition
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Extract the 'PassengerId' column from the test dataset\n
test_ids = test["PassengerId"]

In [36]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
# Remove columns that are not required for analysis to simplify the dataset
def clean(data):
    data = data.drop(["PassengerId","Name", "Ticket", "Cabin"], axis =1)

# Fill missing values in 'Age', 'SibSp', 'Parch', and 'Fare' with the median of their respective columns
    cols = ["Age", "SibSp", "Parch", "Fare"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)

# Fill missing values in 'Embarked' column with 'U' (Unknown)
    data.Embarked.fillna("U", inplace=True)
    return data

# Clean the training and test datasets
data = clean(data)
test = clean(test)


In [38]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [39]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Columns to be label encoded
cols = ["Sex", "Embarked"]
for col in cols:

# Apply Label Encoding to the columns 'Sex' and 'Embarked'
# This converts categorical text data into a model-friendly numerical format
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])

# Print the classes for each column to understand the encoding
print(le.classes_)

# Display the first 5 rows of the cleaned and encoded training dataset
data.head(5)

['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Define the target variable (y) and the feature variables (X)
y = data["Survived"]
X = data.drop("Survived", axis=1)

# Split the dataset into training and validation sets
# X_train and y_train will be used for training the model
# X_val and y_val will be used for evaluating the model's performance
# The test_size parameter specifies the proportion of the data to include in the validation set (here, 20%)
# The random_state parameter ensures reproducibility of the split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [41]:
# Create a Logistic Regression classifier
# The 'random_state' parameter ensures reproducible results
# The 'max_iter' parameter controls the maximum number of iterations for optimization
clf = LogisticRegression(random_state = 0, max_iter = 1000).fit(X_train, y_train)

In [42]:
# Make predictions using the trained Logistic Regression model (clf)
predictions = clf.predict(X_val)

# Calculate the accuracy score to evaluate the model's performance
from sklearn.metrics import accuracy_score

# Display the accuracy of the model on the validation set
accuracy_score(y_val, predictions)

0.8100558659217877

In [43]:
# Make predictions on the test dataset using the trained Logistic Regression model (clf)
submission_preds = clf.predict(test)

In [44]:
# Create a DataFrame to store passenger IDs and their survival predictions
# 'PassengerId' contains the passenger IDs from the test dataset
# 'survived' contains the corresponding survival predictions
df = pd.DataFrame({"PassengerId": test_ids.values,
                   "survived":submission_preds,})